In [2]:
import numpy as np
from scipy.optimize import fsolve, brentq
from scipy.linalg import lu
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 
from math import sin,cos,pi 
from numpy.linalg import lstsq

## 🧩 Task 1: Root-Finding — Solving for Incident Angle \( \theta \)

In this task, we estimate the angle of incidence \( \theta \) from radar phase data using the **Newton-Raphson method**.

### ⚙️ Method:
- **Newton-Raphson algorithm**
- Initial guess: within \([-1.0, 0.0]\)
- Tolerance: \(10^{-6}\)
- True root: \(-0.523598826\)

### ✅ Output:
- Rapid convergence within 5 iterations.
- Final root matches the true value with high accuracy.
- Error tracking table shows decreasing residuals.


In [26]:
def radar_func(theta,C): 
    """
    Menghitung nilai fungsi untuk akar interpretasi dari radar. 
    
    Persamaan f(theta) = C * sin(theta) + pi = 0,
    dimana C = lambda * R / (4 * pi * ht * hr).
    """
    return C * np.sin(theta) + np.pi 
def df_radar_func(theta): 
    return C * cos(theta) 

In [33]:
C = 2 * np.pi 
a_interval = -1.0 
b_interval = 0.0 

tolerance = 1e-6 
max_iterations = 100 
true_root = -0.523598826

In [34]:
def table_open(iterates): 
    rows = [] 
    for i, x in enumerate(iterates): 
        if i == 0: 
            approx_error = None 
        else: 
            approx_error = abs(iterates[i]-iterates[i-1]) / abs(iterates[i]) * 100 
        true_error = abs(x - true_root) / abs(true_root) * 100 
        rows.append({
            "Iteration" : i, 
            "x" : x, 
            "error_a (%)" : approx_error,
            "error_t (&)" : true_error
        })
    return rows 

In [35]:
def newton_raphson(x0): 
    iterates = [x0]
    for i in range(max_iterations): 
        f_val = radar_func(iterates[-1],C)
        df_val = df_radar_func(iterates[-1]) 
        if df_val == 0: 
            raise ZeroDivisionError("Turunan sama dengan 0") 
        x_new = iterates[-1] - f_val / df_val 
        iterates.append(x_new)
        if abs(radar_func(x_new,C)) < tolerance or abs(x_new - iterates[-2]) < tolerance: 
            break
    return iterates 

In [36]:
x0_newton = 0.73 
iters_newton = newton_raphson(x0_newton) 
table_newton = table_open(iters_newton) 
df_newton = pd.DataFrame(table_newton) 

In [37]:
display(df_newton) 

,Iteration,x,error_a (%),error_t (&)
0,0,0.730000,NaN,239.419717
1,1,-0.835901,187.330875,59.645391
2,2,-0.475129,75.931458,9.257052
3,3,-0.522976,9.148920,0.119021
4,4,-0.523599,0.118990,0.000031


## 🧮 Task 2: Solving Linear System — Extracting Path Delays

Here, we solve for signal path delays τ  using a system of equations derived from radar multipath signal components.

## first Method newton-sainder iteration 
    Solves the linear system Ax = b using the Gauss-Seidel iterative method,
    tracking iteration history and approximate error.

    Args:
        A (numpy.ndarray): The coefficient matrix.
        b (numpy.ndarray): The constant vector.
        x_init (numpy.ndarray): The initial guess for the solution vector.
        max_iterations (int): The maximum number of iterations allowed.
        tolerance (float): The convergence tolerance based on max absolute change.

    Returns:
        tuple: A tuple containing:
            - solution (numpy.ndarray): The computed solution vector.
            - iterations_completed (int): The number of iterations performed.
            - combined_df (pandas.DataFrame): DataFrame tracking x values and
                                              approximate error per iteration.

    Raises:
        ValueError: If matrix A is not square or dimensions mismatch.
        ZeroDivisionError: If a zero diagonal element is encountered.

In [39]:
#Function for checking, is matrix diagonallay dominant 

def check_diaDom(A): 
    n = len(A) 
    for i in range (n): 
        diag_val = abs(A[i][i]) #Get absoulute value for diagonal element 
        row_sum = sum(abs(A[i][j]) for j in range(n) if j != i) 
    if diag_val < row_sum:
        #Not diagnallay dominant 
        return False 
    return True 

In [40]:
def GS_method(A, b, x_init, max_iterations=100, tolerance=1e-10):
    n = len(A) 

    if A.shape[0] != n or A.shape[1] != n: 
        raise ValueError("Matrix must be in square") 
    if b.shape[0] != n:
        raise ValueError("Vector b must have the same number of rows as matrix A") 
    if x_init.shape[0] != n: 
        raise ValueError("Initial guess x_init must have the same number of rows as matrix A.")
        
    x = x_init.astype(float).copy() # Ensure float type and create a copy
    history = [x.copy()] # Initialize history list with the initial guess (Iteration 0)
    error_history = []   # Initialize list to store error vectors |x_k - x_{k-1}|
    iterations_completed = 0
    converged = False

    for iteration in range(max_iterations): 
        x_old = x.copy() 

        for i in range(n): 
            # Calculate sum of off-diagonal elements * current x values
            sum1 = np.dot(A[i, :i], x[:i])       # Uses already updated x[j] in this iteration (j < i)
            sum2 = np.dot(A[i, i+1:], x_old[i+1:]) # Uses x values from the previous iteration (j > i)

            # Check for zero diagonal element before division
            diag_element = A[i, i]
            if abs(diag_element) < 1e-15: # Use tolerance for zero check
                raise ZeroDivisionError(f"Near-zero diagonal element A[{i}][{i}] encountered at iteration {iteration + 1}.")

            # Update x[i]
            x[i] = (b[i] - sum1 - sum2) / diag_element

        # Calculate and store approximate error AFTER updating x for this iteration
        current_error = np.abs(x - x_old)
        error_history.append(current_error)

        # Store the state *after* completing this iteration
        history.append(x.copy())
        iterations_completed = iteration + 1

        # Check for convergence using infinity norm (max absolute difference)
        # Note: Using np.linalg.norm(current_error, ord=np.inf) is equivalent
        if np.linalg.norm(x - x_old, ord=np.inf) < tolerance:
            converged = True
            break # Exit loop if converged

    # --- Create DataFrames ---
    df_cols = [f'x{i+1}' for i in range(n)] # Dynamic column names for x values
    history_df = pd.DataFrame(history, columns=df_cols)

    # Error DataFrame (index will correspond to iteration number 1, 2, ...)
    error_cols = [f'error_x{i+1}' for i in range(n)] # Dynamic column names for errors
    # error_history has length = iterations_completed
    error_df = pd.DataFrame(error_history, columns=error_cols)
    # Align error index to start from 1 (error after iter 1, 2, ...)
    error_df.index = pd.RangeIndex(1, len(error_history) + 1)

    # Combine history and error DataFrames
    combined_df = history_df.join(error_df)
    combined_df.index.name = 'Iteration' # Index 0 is initial guess

    if not converged:
        print(f"\nWarning: Gauss-Seidel did not converge within {max_iterations} iterations.")

    return x, iterations_completed, combined_df

In [41]:
A = np.array([[0.95, 0.15, 0.20],
    [0.10, 0.85, 0.25],
    [0.25, 0.30, 0.90]])

b = np.array([1.15, 1.10, 1.30]) 

In [42]:
# Check if the matrix is diagonally dominant
try:
    is_diag_dominant = check_diaDom(A)
    print(f"Is matrix diagonally dominant: {is_diag_dominant}")
except ValueError as e:
    print(f"Error checking diagonal dominance: {e}")
    is_diag_dominant = False # Assume not dominant if check fails

# Initial guess
x_init = np.zeros(len(b))

# --- Run and Display ---
try:
    # Solve using Gauss-Seidel method - capture the combined DataFrame
    solution, iterations, combined_df = GS_method(A, b, x_init, max_iterations=100, tolerance=1e-10)

    print(f"\nProcess stopped after {iterations} iterations.")
    print(f"Final x = {solution}")

    # --- Display Iteration History and Error DataFrame ---
    print("\nIteration History and Approx. Error (x1, x2, x3, error_x1, ...):")
    # Optional: Set display options for better readability
    pd.set_option('display.max_rows', 20)  # Show first/last rows if very long
    pd.set_option('display.float_format', '{:.6f}'.format) # Format floats
    print(combined_df)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.float_format')
    # --- End Display Iteration History ---

    # Verify the solution by calculating the residual
    residual = b - np.dot(A, solution)
    print(f"\nResidual (b - Ax) at final iteration:")
    print(residual)
    residual_norm = np.linalg.norm(residual)
    print(f"Norm of residual: {residual_norm:.6e}") # Scientific notation

except (ZeroDivisionError, ValueError) as e:
    print(f"\nError during Gauss-Seidel execution: {e}")
    print("Cannot proceed with the calculation.")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")

Is matrix diagonally dominant: True

Process stopped after 13 iterations.
Final x = [0.87619435 0.92884733 0.89144135]

Iteration History and Approx. Error (x1, x2, x3, error_x1, ...):
                x1       x2       x3  error_x1  error_x2  error_x3
Iteration                                                         
0         0.000000 0.000000 0.000000       NaN       NaN       NaN
1         1.210526 1.151703 0.724286  1.210526  1.151703  0.724286
2         0.876197 0.978010 0.875053  0.334329  0.173693  0.150767
3         0.871882 0.934175 0.890863  0.004315  0.043835  0.015810
4         0.875475 0.929102 0.891556  0.003593  0.005073  0.000693
5         0.876130 0.928821 0.891468  0.000655  0.000281  0.000088
6         0.876193 0.928840 0.891444  0.000063  0.000019  0.000024
7         0.876195 0.928846 0.891442  0.000002  0.000007  0.000003
8         0.876194 0.928847 0.891441  0.000000  0.000001  0.000000
9         0.876194 0.928847 0.891441  0.000000  0.000000  0.000000
10        0

## Second Method LU Decomposition 

In [43]:
# --- LU Decomposition ---
P, L, U = lu(A)

print("Matrix A:")
print(A)
print("\nPermutation Matrix P:")
print(P)
print("\nLower Triangular Matrix L:")
print(L)
print("\nUpper Triangular Matrix U:")
print(U)

Matrix A:
[[0.95 0.15 0.2 ]
 [0.1  0.85 0.25]
 [0.25 0.3  0.9 ]]

Permutation Matrix P:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Lower Triangular Matrix L:
[[1.         0.         0.        ]
 [0.10526316 1.         0.        ]
 [0.26315789 0.31230284 1.        ]]

Upper Triangular Matrix U:
[[0.95       0.15       0.2       ]
 [0.         0.83421053 0.22894737]
 [0.         0.         0.77586751]]


In [44]:
# --- Verify PA = LU ---
PA = P @ A
LU = L @ U
print("\nVerification: PA and LU difference")
print(PA - LU)

# --- Solve Ly = Pb ---
Pb = np.dot(P, b)
y = np.zeros_like(b)

for i in range(len(y)):
    y[i] = Pb[i] - np.dot(L[i, :i], y[:i])

# --- Solve Ux = y ---
x = np.zeros_like(b)

for i in range(len(x)-1, -1, -1):
    x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]

print("\n✅ Final Solution from LU Decomposition:")
print(f"x1 = {x[0]:.6f}, x2 = {x[1]:.6f}, x3 = {x[2]:.6f}")

# --- Direct Solver Comparison ---
x_direct = np.linalg.solve(A, b)
print("\n✅ Solution from np.linalg.solve:")
print(f"x1 = {x_direct[0]:.6f}, x2 = {x_direct[1]:.6f}, x3 = {x_direct[2]:.6f}")

# --- Compare Difference ---
diff = np.abs(x_direct - x)
print("\n🔍 Absolute difference between LU and Direct method:")
print(diff)

# --- Optional: Residual norm ---
residual = np.linalg.norm(A @ x - b)
print("\n📉 Residual norm (||Ax - b||):")
print(residual)


Verification: PA and LU difference
[[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.38777878e-17 0.00000000e+00 0.00000000e+00]
 [2.77555756e-17 0.00000000e+00 0.00000000e+00]]

✅ Final Solution from LU Decomposition:
x1 = 0.876194, x2 = 0.928847, x3 = 0.891441

✅ Solution from np.linalg.solve:
x1 = 0.876194, x2 = 0.928847, x3 = 0.891441

🔍 Absolute difference between LU and Direct method:
[2.22044605e-16 0.00000000e+00 0.00000000e+00]

📉 Residual norm (||Ax - b||):
2.220446049250313e-16


## 📈 Task 3: Curve Fitting — Modeling (τ(θ))

We now model the functional relationship between incident angle \( \theta \) and delay \( \tau \) using least squares regression.

### 🔧 Model Form:
## τ(θ)=asin(θ)+bθ^2+c

In [45]:
np.random.seed(42) 

theta = np.random.uniform(0,np.pi/2,20) 

theta = np.sort(theta) 

a_true = x_direct[0] 
b_true = x_direct[1]
c_true = x_direct[2]

In [47]:
print(f"a_true : {x_direct[0]}") #Confirm that the a,b,and c true value 

a_true : 0.8761943484448056


### ⚙️ Approach:
- Linear in parameters: fits with ordinary least squares.
- Used 20 values of theta = np.random.uniform(0,np.pi/2,20) 
- True parameters: extracted from Task 2 or manually defined.

In [53]:
# Generate tau values with small random noise
noise = np.random.normal(0, 0.01, len(theta))
tau = a_true * np.sin(theta) + b_true * theta**2 + c_true + noise

# Step 2: Construct matrix A for linear least squares
A = np.column_stack([np.sin(theta), theta**2, np.ones_like(theta)])

# Step 3: Solve A p = tau using least squares
p_est, residuals, rank, s = lstsq(A, tau, rcond=None)

# Predicted tau using fitted model
tau_fit = A @ p_est

# Calculate R-squared
ss_total = np.sum((tau - np.mean(tau))**2)
ss_residual = np.sum((tau - tau_fit)**2)
r_squared = 1 - (ss_residual / ss_total)

# Calculate mean squared error
mse = np.mean((tau - tau_fit)**2)

# Create table to display
df_Display = pd.DataFrame({
    "θ (rad)": theta,
    "sin(θ)": np.sin(theta),
    "θ²": theta**2,
    "τ (observed)": tau,
    "τ (fitted)": tau_fit
})

In [54]:
display(df_Display) 

,θ (rad),sin(θ),θ²,τ (observed),τ (fitted)
0,0.032334,0.032328,0.001045,0.923700,0.919243
1,0.091238,0.091111,0.008324,0.981615,0.977492
2,0.245036,0.242591,0.060042,1.159820,1.158308
3,0.245074,0.242628,0.060061,1.157472,1.158357
4,0.285610,0.281743,0.081573,1.199918,1.212648
5,0.288091,0.284123,0.082996,1.213273,1.216057
6,0.333541,0.327391,0.111250,1.278207,1.280266
7,0.457462,0.441672,0.209271,1.464790,1.471652
8,0.477903,0.459918,0.228391,1.504946,1.505441
9,0.588326,0.554969,0.346128,1.703243,1.698360


In [55]:
print(f"Estimated parameters: a={p_est[0]:.4f}, b={p_est[1]:.4f}, c={p_est[2]:.4f}")

Estimated parameters: a=0.8756, b=0.9317, c=0.8900


### 📊 Metrics:
- R^2 > 0.9999 
- MSE 1e-4
- Curve matches observed data almost perfectly.

In [50]:
print(f"True parameters: a={a_true:.4f}, b={b_true:.4f}, c={c_true:.4f}")
print(f"R-squared: {r_squared:.6f}")
print(f"Mean Squared Error: {mse:.6f}")

True parameters: a=0.8762, b=0.9288, c=0.8914
R-squared: 0.999930
Mean Squared Error: 0.000066
